### Vectorize Data Sets

In [1]:
from numba import jit, autojit # accelerate for loops
import numpy as np
import numpy.random as npr
import re # regular expressions
import glob
import pandas as pd

In [2]:
###--- Methods ---###

#### bootstrapping: resampling (virtual example)

nb = 100  # how many samples ?
ln = 3000 # how many words for each sample ? (3000 ~ length of each text)

def btp_ori(tot,nb,ln):
    if len(tot) == 0: return np.empty(0)
    else:
        idx = npr.randint(0, len(tot), (nb,ln)) # randomly pick indices from the 'at' array, 'nb' rows
        return np.array(tot[idx])

#### extract function words
# function word list: Bei Yu 2012: Function Words for Chinese Authorship Attribution
fw1 = ["的","是","不","了","在","有","這","為","地","也","得","就","那","以","著","之","可","于"
       "麼","而","然","没","於","還","只","無","又","如","但","其","此","與","把","全","被","卻"]

# difficult word list: 司法院裁判書類通俗化小組
fw2 = ["殆","似","難認無過失","難認有理由","尚難謂為於法無違","非無理由","非有理由","尚非無疑","尚非不可能",
       "可徵","徵諸","洵堪","迭於","係經","係指","洵屬","伊","渠","致","爰","遽認","矧","略以","按","第按","是","茲",
       "揆諸","皆無該條適用","即","無庸","縱",
       "上開","足資","可稽","堪以認定","業據","云云","等語","之","次查","復查","再查","系爭","固非無見","旋","惟查","均係"]
fwt = fw1+fw2

@jit
def ext_fw(txt,fw):
    # txt = input texts; fw = function word list
    j = np.empty(0)
    for i in fw:
        l = re.findall(i,txt)
        j = np.concatenate((j,np.array(l)))
    return j

In [15]:
#### generate csv files
global file_idx
file_idx = 0
def fw_csv(path):
    list_of_files = glob.glob(path)

    for file_n in list_of_files:
        # print file_n
        #at = np.empty(0) # bootstrapping for EACH file
        with open(file_n,'r') as myfile:         
            sl = myfile.read().replace('\n','')   # returns a string
            at = np.array(re.findall(r'\S+', sl)) # returns an array of the above string
            
            # all texts as one
            #at = np.concatenate((at,sa))
        
        btp = btp_ori(at,nb,ln) # virtual examples: # = nb, length = ln

        for i in range(btp.shape[0]):
            bts = " ".join(btp[i])
            fwr = ext_fw(bts,fw1+fw2)

            # frequency table
            fwr_str = "\t".join(fwr).split("\t")
            fwr_tb  = pd.Series(fwr_str).value_counts()

            # store in dictionary
            dict1 = {}
            dict1.update(fwr_tb)
            dict1.update({"NAME": "%s" %f})
            dict1.update({"file_n": file_idx})
            tp.append(dict1)
        global file_idx
        file_idx+=1
       
    return tp
    

<ipython-input-15-a434e7347374>:33: SyntaxWarning: name 'file_idx' is used prior to global declaration
  global file_idx


In [8]:
#### read corpus
list_of_jud   = ["翁岳生","城仲模","林永謀","王和雄","余雪明","廖義男","曾有田","楊仁壽",
                 "彭鳳至","賴英照","謝在全","徐璧湖","林子儀","許宗力","許玉秀","林錫堯",
                 "池啟明","李震山","蔡清遊","賴浩敏","蘇永欽","黃茂榮","陳新民","陳春生","陳敏","葉百修"]

tp = [] # an empty list
file_idx = 0
for f in list_of_jud:
    # for each author ...
    path = './segmented_txt_2/%s/*.txt' %f 
    rs   = fw_csv(path)
    
df = pd.DataFrame(rs) 
# output csv
df.to_csv("./vectorized.csv", encoding="utf-8")

---
### Examine results:

In [9]:
import pandas as pd
test = pd.read_csv("./vectorized.csv", encoding="utf-8")
test

,Unnamed: 0,NAME,file_n,上開,不,之,也,了,云云,以,...,致,與,茲,著,被,足資,這,遽認,還,那
0,0,翁岳生,0,1,67,340,NaN,NaN,NaN,43,...,3,32,NaN,4,1,NaN,NaN,NaN,NaN,NaN
1,1,翁岳生,0,3,49,350,NaN,NaN,NaN,43,...,4,34,NaN,1,2,NaN,NaN,NaN,NaN,NaN
2,2,翁岳生,0,1,52,374,NaN,NaN,NaN,40,...,5,24,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
3,3,翁岳生,0,NaN,48,340,NaN,NaN,NaN,35,...,2,54,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN
4,4,翁岳生,0,1,60,400,NaN,NaN,NaN,41,...,1,36,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
5,5,翁岳生,0,2,53,348,NaN,NaN,NaN,40,...,6,34,NaN,1,1,NaN,NaN,NaN,NaN,NaN
6,6,翁岳生,0,2,46,370,NaN,NaN,NaN,39,...,2,30,NaN,3,1,NaN,NaN,NaN,NaN,NaN
7,7,翁岳生,0,2,55,346,NaN,NaN,NaN,40,...,2,26,NaN,3,2,NaN,NaN,NaN,NaN,NaN
8,8,翁岳生,0,2,48,348,NaN,NaN,NaN,42,...,NaN,41,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
9,9,翁岳生,0,5,45,358,NaN,NaN,NaN,52,...,4,46,NaN,1,1,NaN,NaN,NaN,NaN,NaN


---
### Generate testing data from judicial Yuan (cf. parser.ipynb)

In [16]:
tp = [] # an empty list
path = './test_d/*.txt'
rs   = fw_csv(path)
f    = "test data"
    
df = pd.DataFrame(rs) 
# output csv
df.to_csv("./test_ds.csv", encoding="utf-8")

In [17]:
import pandas as pd
test = pd.read_csv("./test_ds.csv", encoding="utf-8")
test

,Unnamed: 0,Unnamed: 1,NAME,file_n,上開,不,之,了,云云,以,...,系爭,縱,而,致,與,茲,著,被,足資,還
0,0,NaN,test data,0,NaN,39,184,NaN,NaN,45,...,8,NaN,10,7,14,NaN,3,NaN,NaN,NaN
1,1,NaN,test data,0,NaN,29,210,NaN,NaN,48,...,8,NaN,9,1,10,NaN,5,NaN,NaN,NaN
2,2,NaN,test data,0,NaN,33,206,NaN,NaN,44,...,6,NaN,9,NaN,12,NaN,7,NaN,NaN,NaN
3,3,NaN,test data,0,NaN,29,200,NaN,NaN,53,...,2,NaN,8,3,13,NaN,5,NaN,NaN,NaN
4,4,NaN,test data,0,NaN,30,208,NaN,NaN,37,...,8,NaN,4,7,13,NaN,3,NaN,NaN,NaN
5,5,NaN,test data,0,NaN,34,188,NaN,NaN,52,...,7,NaN,5,6,11,NaN,2,NaN,NaN,NaN
6,6,NaN,test data,0,NaN,36,204,NaN,NaN,61,...,7,NaN,3,12,16,NaN,3,NaN,NaN,NaN
7,7,NaN,test data,0,NaN,40,178,NaN,NaN,57,...,7,NaN,6,2,13,NaN,6,NaN,NaN,NaN
8,8,NaN,test data,0,NaN,40,172,NaN,NaN,47,...,4,NaN,13,4,18,NaN,4,NaN,NaN,NaN
9,9,NaN,test data,0,NaN,43,214,NaN,NaN,49,...,5,NaN,10,2,21,NaN,7,NaN,NaN,NaN
